In [1]:
import os
from pathlib import Path
import pyvista as pv
import numpy as np
from scipy.spatial import KDTree

In [2]:
root_dir = Path("/home/max/Desktop/python_projects/SimJeb/")
metadata_dir = root_dir / "SimJEB_metadata"
meshes_dir = root_dir / "SimJEB_surfmesh"

In [3]:
pc = pv.read(metadata_dir / "train_volmesh_points_density.vtk")

In [4]:
n_cells = 500

In [17]:
bbox = np.reshape(pc.bounds, (3,2))
dims = bbox[:,1] - bbox[:,0]
coeff = (n_cells / np.prod(dims)) ** (1/3)
bins = (dims * coeff).astype(np.int32)
if bins[1] % 2 == 0:
    bins[1] += 1
H, edges = np.histogramdd(pc.points, bins=bins)
centers = [(e[1:] + e[:-1]) / 2 for e in edges]
x, y, z = np.meshgrid(centers[0], centers[1], centers[2], indexing="ij")
centroids_hist = np.c_[x.reshape(-1), y.reshape(-1), z.reshape(-1)]

In [18]:
bins

array([ 8, 13,  4], dtype=int32)

In [19]:
kdt = KDTree(centroids_hist)

In [20]:
spacing = dims / bins
grid = pv.ImageData()
grid.dimensions = bins + 1
grid.spacing = spacing
grid.origin = bbox[:,0]
centroids_grid = grid.cell_centers().points
_, ii = kdt.query(centroids_grid)
num_points = np.array(H).flatten()[ii]
grid["num_points"] = num_points

In [21]:
grid.save(metadata_dir / "density_grid.vtk")

In [22]:
nonzero_num_points = num_points[num_points > 0]
q = 0.7
grid_interfaces = grid.threshold(np.quantile(nonzero_num_points, q), invert=False)
grid_rare = grid.threshold(np.quantile(nonzero_num_points, q), invert=True).threshold(1e-5, invert=False)
grid_interfaces.save(metadata_dir / "grid_interfaces.vtk")
grid_rare.save(metadata_dir / "grid_rare.vtk")

In [23]:
grid.n_cells, grid_rare.n_cells, grid_interfaces.n_cells,

(416, 199, 86)